# Sleep Disorder Classification — Training with Checkpoints & Confusion Matrices

**Loads preprocessed `.npy` files from Google Drive, trains 5 models with full diagnostics.**

Features:
- Loads saved `X_train_full.npy`, `X_test.npy`, etc. (no re-preprocessing)
- Generates **separate confusion matrices** for Train / Validation / Test per model
- Saves model weights as **numbered checkpoints** for iterative improvement

> **Runtime**: Set to **T4 GPU** via Runtime → Change runtime type

In [ ]:
# 1. Setup
!pip install -q optuna imbalanced-learn seaborn

import torch
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB)')
else:
    print('WARNING: No GPU! Go to Runtime -> Change runtime type -> T4 GPU')

In [ ]:
# 2. Mount Drive & Load Preprocessed Data
from google.colab import drive
import numpy as np
import joblib
import os

drive.mount('/content/drive')

DATA_DIR = '/content/drive/MyDrive/Sleep_Disorder_Project'
CKPT_DIR = os.path.join(DATA_DIR, 'checkpoints')
os.makedirs(CKPT_DIR, exist_ok=True)

X_train_full = np.load(os.path.join(DATA_DIR, 'X_train_full.npy'))
y_train_full = np.load(os.path.join(DATA_DIR, 'y_train_full.npy'))
X_test = np.load(os.path.join(DATA_DIR, 'X_test.npy'))
y_test = np.load(os.path.join(DATA_DIR, 'y_test.npy'))
le = joblib.load(os.path.join(DATA_DIR, 'label_encoder.joblib'))

print(f'Train: {X_train_full.shape}, Test: {X_test.shape}')
print(f'Classes: {list(le.classes_)}')

In [ ]:
# 3. Create Train/Val Split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.15, random_state=42, stratify=y_train_full
)
print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')

In [ ]:
# 4. Confusion Matrix Plotting Utility
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

def plot_confusion_matrices(model_name, y_sets, pred_sets, class_names, save_dir=None):
    """
    Plot side-by-side confusion matrices for train/val/test.
    y_sets: dict of {'Train': y_train, 'Val': y_val, 'Test': y_test}
    pred_sets: dict of {'Train': preds_train, 'Val': preds_val, 'Test': preds_test}
    """
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    fig.suptitle(f'{model_name} — Confusion Matrices', fontsize=16, fontweight='bold')

    for ax, split_name in zip(axes, ['Train', 'Val', 'Test']):
        y_true = y_sets[split_name]
        y_pred = pred_sets[split_name]
        cm = confusion_matrix(y_true, y_pred)
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')

        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names,
                    yticklabels=class_names, ax=ax)
        ax.set_title(f'{split_name}\nAcc: {acc:.4f} | F1: {f1:.4f}')
        ax.set_ylabel('True')
        ax.set_xlabel('Predicted')

    plt.tight_layout()
    if save_dir:
        path = os.path.join(save_dir, f'{model_name}_confusion_matrices.png')
        plt.savefig(path, dpi=150, bbox_inches='tight')
        print(f'Saved: {path}')
    plt.show()

    # Print classification reports
    for split_name in ['Train', 'Val', 'Test']:
        print(f"\n--- {model_name} {split_name} Classification Report ---")
        print(classification_report(y_sets[split_name], pred_sets[split_name], target_names=class_names))

In [ ]:
# 5. Checkpoint Utilities
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import glob

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
USE_AMP = torch.cuda.is_available()
BATCH_SIZE = 512 if torch.cuda.is_available() else 64

class SleepDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]


def get_next_checkpoint_num(model_name, ckpt_dir):
    """Find the next checkpoint number for a model."""
    existing = glob.glob(os.path.join(ckpt_dir, f'{model_name}_v*.pth')) + \
               glob.glob(os.path.join(ckpt_dir, f'{model_name}_v*.joblib'))
    if not existing:
        return 1
    nums = []
    for f in existing:
        base = os.path.basename(f)
        try:
            num = int(base.split('_v')[1].split('.')[0])
            nums.append(num)
        except:
            pass
    return max(nums) + 1 if nums else 1


def save_checkpoint(model, model_name, ckpt_dir, is_pytorch=False, metrics=None):
    """Save model with versioned checkpoint name."""
    ver = get_next_checkpoint_num(model_name, ckpt_dir)
    if is_pytorch:
        path = os.path.join(ckpt_dir, f'{model_name}_v{ver}.pth')
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'version': ver,
            'metrics': metrics or {}
        }
        torch.save(checkpoint, path)
    else:
        path = os.path.join(ckpt_dir, f'{model_name}_v{ver}.joblib')
        joblib.dump({'model': model, 'version': ver, 'metrics': metrics or {}}, path)
    print(f'Checkpoint saved: {path} (v{ver})')
    return path


def make_loaders(*args_pairs, batch_size=BATCH_SIZE):
    """Create DataLoaders from (X, y) pairs."""
    pin = torch.cuda.is_available()
    loaders = []
    for X, y in args_pairs:
        ds = SleepDataset(X, y)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, pin_memory=pin, num_workers=2)
        loaders.append(dl)
    return loaders


def predict_torch(model, loader):
    """Get predictions from a PyTorch model."""
    model.eval()
    preds = []
    with torch.no_grad(), autocast(enabled=USE_AMP):
        for Xb, _ in loader:
            out = model(Xb.to(DEVICE, non_blocking=True))
            preds.extend(torch.max(out, 1)[1].cpu().numpy())
    return np.array(preds)

print(f'Device: {DEVICE} | Batch: {BATCH_SIZE} | AMP: {USE_AMP}')

In [ ]:
# 6. Model Definitions
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

class ANN(nn.Module):
    def __init__(self, input_dim, hidden_layers=2, units=128, dropout=0.3, num_classes=3):
        super().__init__()
        layers = [nn.Linear(input_dim, units), nn.BatchNorm1d(units), nn.ReLU(), nn.Dropout(dropout)]
        for _ in range(hidden_layers - 1):
            layers += [nn.Linear(units, units), nn.BatchNorm1d(units), nn.ReLU(), nn.Dropout(dropout)]
        layers.append(nn.Linear(units, num_classes))
        self.network = nn.Sequential(*layers)
    def forward(self, x): return self.network(x)

class CNN(nn.Module):
    def __init__(self, input_dim, filters=32, kernel_size=2, dropout=0.3, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(1, filters, kernel_size)
        self.bn1 = nn.BatchNorm1d(filters)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        conv_out = input_dim - kernel_size + 1
        pool_out = conv_out // 2
        self.pool = nn.MaxPool1d(2) if pool_out > 0 else nn.Identity()
        if pool_out <= 0: pool_out = conv_out
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(filters * pool_out, num_classes)
    def forward(self, x):
        x = self.dropout(self.pool(self.relu(self.bn1(self.conv1(x.unsqueeze(1))))))
        return self.fc(self.flatten(x))

INPUT_DIM = X_train.shape[1]
NUM_CLASSES = len(le.classes_)
print(f'Input dim: {INPUT_DIM}, Classes: {NUM_CLASSES}')

In [ ]:
# 7. PyTorch Training with Checkpointing
def train_pytorch_model(model, model_name, X_tr, y_tr, X_v, y_v, epochs=20, lr=0.001):
    """Train PyTorch model with AMP, return predictions on all sets."""
    model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scaler = GradScaler(enabled=USE_AMP)

    train_ds = SleepDataset(X_tr, y_tr)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          pin_memory=torch.cuda.is_available(), num_workers=2)

    best_f1 = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for Xb, yb in train_dl:
            Xb, yb = Xb.to(DEVICE, non_blocking=True), yb.to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=USE_AMP):
                loss = criterion(model(Xb), yb)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        # Quick val check
        val_dl = make_loaders((X_v, y_v))[0]
        val_preds = predict_torch(model, val_dl)
        val_f1 = f1_score(y_v, val_preds, average='weighted')

        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f'  Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_dl):.4f} | Val F1: {val_f1:.4f}')

        if val_f1 > best_f1:
            best_f1 = val_f1

    return model

---
## Train & Evaluate All Models
Each model is trained, evaluated on Train/Val/Test, confusion matrices are plotted, and checkpoints are saved.

In [ ]:
# 8a. KNN
print('='*50)
print('Training KNN...')
print('='*50)

knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='euclidean', n_jobs=-1)
knn.fit(X_train, y_train)

knn_preds = {
    'Train': knn.predict(X_train),
    'Val':   knn.predict(X_val),
    'Test':  knn.predict(X_test)
}

plot_confusion_matrices('KNN',
    {'Train': y_train, 'Val': y_val, 'Test': y_test},
    knn_preds, le.classes_, save_dir=CKPT_DIR)

save_checkpoint(knn, 'KNN', CKPT_DIR, is_pytorch=False,
    metrics={'test_acc': accuracy_score(y_test, knn_preds['Test']),
             'test_f1': f1_score(y_test, knn_preds['Test'], average='weighted')})

In [ ]:
# 8b. SVM
print('='*50)
print('Training SVM...')
print('='*50)

# SVM on subsample for speed
svm_sample = min(len(X_train), 20000)
idx = np.random.choice(len(X_train), size=svm_sample, replace=False)
svm = SVC(C=10.0, kernel='rbf', gamma='scale')
svm.fit(X_train[idx], y_train[idx])

svm_preds = {
    'Train': svm.predict(X_train),
    'Val':   svm.predict(X_val),
    'Test':  svm.predict(X_test)
}

plot_confusion_matrices('SVM',
    {'Train': y_train, 'Val': y_val, 'Test': y_test},
    svm_preds, le.classes_, save_dir=CKPT_DIR)

save_checkpoint(svm, 'SVM', CKPT_DIR, is_pytorch=False,
    metrics={'test_acc': accuracy_score(y_test, svm_preds['Test']),
             'test_f1': f1_score(y_test, svm_preds['Test'], average='weighted')})

In [ ]:
# 8c. Random Forest
print('='*50)
print('Training Random Forest...')
print('='*50)

rf = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

rf_preds = {
    'Train': rf.predict(X_train),
    'Val':   rf.predict(X_val),
    'Test':  rf.predict(X_test)
}

plot_confusion_matrices('RF',
    {'Train': y_train, 'Val': y_val, 'Test': y_test},
    rf_preds, le.classes_, save_dir=CKPT_DIR)

save_checkpoint(rf, 'RF', CKPT_DIR, is_pytorch=False,
    metrics={'test_acc': accuracy_score(y_test, rf_preds['Test']),
             'test_f1': f1_score(y_test, rf_preds['Test'], average='weighted')})

In [ ]:
# 8d. ANN
print('='*50)
print('Training ANN...')
print('='*50)

ann = ANN(INPUT_DIM, hidden_layers=2, units=128, dropout=0.3, num_classes=NUM_CLASSES)
ann = train_pytorch_model(ann, 'ANN', X_train, y_train, X_val, y_val, epochs=20, lr=0.001)

train_dl, val_dl, test_dl = make_loaders((X_train, y_train), (X_val, y_val), (X_test, y_test))
ann_preds = {
    'Train': predict_torch(ann, train_dl),
    'Val':   predict_torch(ann, val_dl),
    'Test':  predict_torch(ann, test_dl)
}

plot_confusion_matrices('ANN',
    {'Train': y_train, 'Val': y_val, 'Test': y_test},
    ann_preds, le.classes_, save_dir=CKPT_DIR)

save_checkpoint(ann, 'ANN', CKPT_DIR, is_pytorch=True,
    metrics={'test_acc': accuracy_score(y_test, ann_preds['Test']),
             'test_f1': f1_score(y_test, ann_preds['Test'], average='weighted')})

In [ ]:
# 8e. CNN
print('='*50)
print('Training CNN...')
print('='*50)

cnn = CNN(INPUT_DIM, filters=32, kernel_size=2, dropout=0.3, num_classes=NUM_CLASSES)
cnn = train_pytorch_model(cnn, 'CNN', X_train, y_train, X_val, y_val, epochs=20, lr=0.001)

cnn_preds = {
    'Train': predict_torch(cnn, train_dl),
    'Val':   predict_torch(cnn, val_dl),
    'Test':  predict_torch(cnn, test_dl)
}

plot_confusion_matrices('CNN',
    {'Train': y_train, 'Val': y_val, 'Test': y_test},
    cnn_preds, le.classes_, save_dir=CKPT_DIR)

save_checkpoint(cnn, 'CNN', CKPT_DIR, is_pytorch=True,
    metrics={'test_acc': accuracy_score(y_test, cnn_preds['Test']),
             'test_f1': f1_score(y_test, cnn_preds['Test'], average='weighted')})

In [ ]:
# 9. Final Summary
import pandas as pd

summary = {}
all_preds = {'KNN': knn_preds, 'SVM': svm_preds, 'RF': rf_preds, 'ANN': ann_preds, 'CNN': cnn_preds}
all_y = {'Train': y_train, 'Val': y_val, 'Test': y_test}

for model_name, preds in all_preds.items():
    for split in ['Train', 'Val', 'Test']:
        acc = accuracy_score(all_y[split], preds[split])
        f1 = f1_score(all_y[split], preds[split], average='weighted')
        summary[f'{model_name}_{split}'] = {'Accuracy': acc, 'F1': f1}

df = pd.DataFrame(summary).T
print('\n' + '='*60)
print('FULL RESULTS SUMMARY (all splits)')
print('='*60)
print(df.to_string(float_format='%.4f'))

# Highlight overfitting detection
print('\n--- Overfitting Check (Train F1 - Test F1) ---')
for m in ['KNN', 'SVM', 'RF', 'ANN', 'CNN']:
    train_f1 = summary[f'{m}_Train']['F1']
    test_f1 = summary[f'{m}_Test']['F1']
    gap = train_f1 - test_f1
    status = 'OK' if gap < 0.05 else 'OVERFIT' if gap > 0.1 else 'MODERATE'
    print(f'  {m:4s}: Train={train_f1:.4f}  Test={test_f1:.4f}  Gap={gap:.4f}  [{status}]')

In [ ]:
# 10. List All Checkpoints
print('\nSaved checkpoints:')
for f in sorted(os.listdir(CKPT_DIR)):
    path = os.path.join(CKPT_DIR, f)
    size = os.path.getsize(path) / (1024*1024)
    print(f'  {f} ({size:.1f} MB)')